In [ ]:
!pip install -q datasets
!huggingface-cli login

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_| 

In [ ]:
from datasets import load_dataset
import re

# Load the dataset
dataset = load_dataset("Cynaptics/persona-chat")

# Shuffle the dataset and slice it
dataset = dataset['train'].shuffle(seed=42).select(range(1000))

# Define a function to transform the data
def transform_conversation(example):
    conversation_text = example['dialogue']  # Accessing the 'dialogue' field
    persona_b_facts = example['persona_b']  # Accessing the 'persona_b' field

    # Create a string from persona_b facts (if needed for context)
    persona_b_context = " ".join(persona_b_facts)  # Combine facts into a single string

    # Split dialogue into individual turns
    reformatted_segments = []
    for i in range(0, len(conversation_text), 2):  # Iterating with step 2 to get Persona A and B turns
        human_text = conversation_text[i].strip()  # Persona A's turn
        assistant_text = conversation_text[i + 1].strip() if i + 1 < len(conversation_text) else ""  # Persona B's turn

        # Apply the new template, considering persona_b context
        if assistant_text:
            reformatted_segments.append(f'<s>[INST] {human_text} </INST> {assistant_text} </s>')
        else:
            reformatted_segments.append(f'<s>[INST] {human_text} </INST> </s>')

    # Add the persona_b context at the start (if needed)
    # This will make the conversation more personalized by including Persona B's background
    return {'text': f"<persona_b>{persona_b_context}</persona_b>" + ''.join(reformatted_segments)}

# Apply the transformation
transformed_dataset = dataset.map(transform_conversation)

# Print a few transformed examples for validation
for example in transformed_dataset.select(range(5)):  # Selecting the first 5 examples
    print(example['text'])


<persona_b>I don't really care about the weather. I have a love for travel and my furry friends. i am a carnivore. I started a blog to share my love of romance novels with others. I had a small party with my friends and family at the hotel where I was staying.</persona_b><s>[INST] Persona A: Here, welcome! </INST> Persona B: How are you doing today? </s><s>[INST] Persona A: I am doing well, yesterday i spent hours planning my next garden project. </INST> Persona B: That sounds like fun! I am not really good with plants, but animals have always been my passion. </s><s>[INST] Persona A: I have a dog and cat too! </INST> Persona B: What are their names? </s><s>[INST] Persona A: My dog is named Buddy and my cat has been called Mittens. </INST> Persona B: These are great names! I have one dog named Bailey and another cat named Shadow. </s><s>[INST] Persona A: That’s a great name for one dog! I love the color of Bailey' fur. </INST> Persona B: Thank you! I love my pets they are some of the b

In [ ]:
transformed_dataset.push_to_hub("llama2-1k")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/rudrajadon18/guanaco-llama2-1k/commit/236e11f99f364bdac5ea8708af524d4dcb83263b', commit_message='Upload dataset', commit_description='', oid='236e11f99f364bdac5ea8708af524d4dcb83263b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/rudrajadon18/guanaco-llama2-1k', endpoint='https://huggingface.co', repo_type='dataset', repo_id='rudrajadon18/guanaco-llama2-1k'), pr_revision=None, pr_num=None)